## Average hash
1. 이미지 크기를 8X8
2. grey scale
3. 각 픽셀의 평균 계산
4. 각 픽셀의 어두운 정도가 평균보다 크면 1, 아니면 0

In [2]:
from PIL import Image
import numpy as np

#### 이미지 데이터를 average hash로 변환

In [3]:
def average_hash(fname, size = 16) :
    img = Image.open(fname) #image 데이터 열기
    img = img.convert('L') #grey scale
    img = img.resize((size,size), Image.ANTIALIAS) #anti alias
    
    pixel_data = img.getdata() #픽셀 가져오기
    pixels = np.array(pixel_data)
    pixels = pixels.reshape((size,size))
    
    avg = pixels.mean()
    
    diff = 1*(pixels > avg) #평균보다 크면 1, 아니면 0
    
    return diff

In [10]:
def np2hash(n) :
    bhash = []
    for nl in ahash.tolist() :
        sl = [str(i) for i in nl]
        s2 = "".join(sl)
        i = int(s2,2)
        bhash.append("%04x"% i)
    
    return ("".join(bhash))

In [6]:
path = "C:/Users/wjssm/Desktop/[스터디] 파이썬_머신러닝/ex_code/ch7/photo/"

In [8]:
ahash = average_hash(path+'tower.jpg')
print(ahash)

[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 1 1 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 1 1 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 1 1 1 0 0 0 0 0 0]
 [1 0 0 0 0 1 1 1 1 1 1 0 0 0 0 0]
 [0 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0]
 [0 1 0 1 1 1 1 1 1 1 1 1 0 0 0 0]
 [0 1 1 1 1 1 1 1 1 1 1 1 0 0 1 0]
 [1 1 0 0 1 1 1 1 1 1 1 1 1 0 1 0]
 [1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 0]
 [0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0]
 [0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0]
 [0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1]]


In [11]:
print(np2hash(ahash))

000001000180018003c003c003c087e07ff05ff07ff2cffa9ffe3ffe3ffe1fff


# average hash를 사용해 유사한 이미지 검색하는 도구

In [12]:
import os,re

In [44]:
search_dir = "./data/image/101_ObjectCategories"
cache_dir = "./data/image/cache_avhash"

In [47]:
if not os.path.exists(cache_dir):
    os.mkdir(cache_dir)

#### 이미지 데이터를 average hash로 변환

In [48]:
def average_hash(fname, size = 16) :
    fname2 = fname[len(search_dir):]
    # 이미지 캐시하기
    cache_file = cache_dir + "/" + fname2.replace('/', '_') + ".csv"
    
    if not os.path.exists(cache_file) :#해시 생성하기
        img = Image.open(fname)
        img = img.convert('L').resize((size,size), Image.ANTIALIAS)
        pixels = np.array(img.getdata()).reshape((size,size))
        avg = pixels.mean()
        px = 1*(pixels>avg)
        np.savetxt(cache_file, px,fmt = "%.0f", delimiter=",")
    else :
        px = np.loadtxt(cache_file, delimiter=",")
        
    return px

In [49]:
def hamming_dist(a,b) :
    aa = a.reshape(1,-1) #1차원 배열로
    ab = b.reshape(1,-1)
    dist = (aa!=ab).sum #aa와 ab 중 다른 것 개수
    
    return dist

#### 모든 폴더에 처리 적용하기

In [50]:
def enum_all_files(path):
    for root, dirs, files in os.walk(path):
        for f in files:
            fname = os.path.join(root, f)
            if re.search(r'\.(jpg|jpeg|png)$', fname):
                yield fname

#### 이미지 찾기

In [53]:
def find_image(fname, rate) :
    src = average_hash(fname)
    
    for fname in enum_all_files(search_dir) :
        dst = average_hash(fname)
        diff_r = hamming_dist(src,dst) / 256
        
        if diff_r<rate :
            yield(diff_r, fname)

In [54]:
srcfile = search_dir + "/chair/image_0016.jpg"
html = ""
sim = list(find_image(srcfile, 0.25))
sim = sorted(sim, key=lambda x:x[0])
for r, f in sim:
    print(r, ">", f)
    s = '<div style="float:left;"><h3>[ 차이 :' + str(r) + '-' + \
        os.path.basename(f) + ']</h3>'+ \
        '<p><a href="' + f + '"><img src="' + f + '" width=400>'+ \
        '</a></p></div>'
    html += s

FileNotFoundError: [Errno 2] No such file or directory: './data/image/cache_avhash/\\accordion\\image_0001.jpg.csv'